# Meters Case Study

This notebook contains calculations for the meters case study

- **Assumptions**
  - According to profileSolar.com, the solar irradiance in Christchurch, NZ is 4kWh/m²/day. This will be used to calculate the daily production by the solar panels.

- **Daily Production**
  - Daily Production = Solar Irradiance * Solar Capacity = 40kW × 4kWh/m²/day = 160kWh/day

- **Trading Periods**
  - Since there are 48 trading periods in a day (30-minute intervals):
    - Production per trading period = 160 / 48 ≈ 3.33kWh/trading period

- **Revenue Calculation**
  - Revenue = Energy used * Price per kWh

In [1]:
#Import relevant libraries
import pandas as pd

In [14]:
#First meter
data = pd.read_csv('NewMeterWorkbook - 216298730.csv')
data.head()

,CUST,Start time for each 30min measurement period,Consumption in kWh,Trading Period,Price $/MWh for each trading period,Solar Energy Used,Excess Energy Exported to Grid,Revenue from Energy Used by Building,Revenue from Energy Exported to Grid,Total Revenue per Period,Unnamed: 10
0,216298730.0,2023-01-01 00:00:00,0.7,1.0,3.24,0.7,2.63,0.203,0.3156,0.5186,NaN
1,216298730.0,2023-01-01 00:30:00,0.2,2.0,4.36,0.2,3.13,0.058,0.3756,0.4336,NaN
2,216298730.0,2023-01-01 01:00:00,0.4,3.0,3.61,0.4,2.93,0.116,0.3516,0.4676,NaN
3,216298730.0,2023-01-01 01:30:00,1.2,4.0,6.13,1.2,2.13,0.348,0.2556,0.6036,NaN
4,216298730.0,2023-01-01 02:00:00,0.2,5.0,4.74,0.2,3.13,0.058,0.3756,0.4336,NaN


#  Meter 1

In [15]:
import pandas as pd



# Step 2: Convert time column to datetime format for better handling
data['Start time for each 30min measurement period'] = pd.to_datetime(
    data['Start time for each 30min measurement period']
)

# Constants for calculations
battery_capacity_kWh = 61  # Total battery capacity in kWh
battery_discharge_limit_kWh = battery_capacity_kWh * 0.6  # 60% discharge capacity
grid_price_cents = 12  # Price for selling energy to the grid (12 cents/kWh)
building_price_cents = 29  # Price for selling energy to the building (29 cents/kWh)

# Convert spot price from $/MWh to cents/kWh for easier comparison
data['Spot Price (cents/kWh)'] = data['Price $/MWh for each trading period'] / 10

# ------------------------------
# SCENARIO 1: Revenue Calculation
# ------------------------------

# Step 1: Calculate total revenue from solar energy used by the building
total_solar_used_revenue = data['Revenue from Energy Used by Building'].sum()

# Step 2: Calculate total revenue from excess solar energy exported to the grid
total_grid_export_revenue = data['Revenue from Energy Exported to Grid'].sum()

# Step 3: Combine these to get total revenue for Scenario 1
total_revenue_scenario_1 = total_solar_used_revenue + total_grid_export_revenue

# Step 4: Adjust for the impact of the 61kWh battery
# Peak times are defined as 6-9 AM and 6-9 PM (morning and evening periods)
peak_hours_morning = data[(data['Start time for each 30min measurement period'].dt.hour >= 6) &
                          (data['Start time for each 30min measurement period'].dt.hour < 9)]
peak_hours_evening = data[(data['Start time for each 30min measurement period'].dt.hour >= 18) &
                          (data['Start time for each 30min measurement period'].dt.hour < 21)]

# Calculate how much battery energy is available per peak period
total_peak_periods = len(peak_hours_morning) + len(peak_hours_evening)
battery_discharge_per_period_kWh = battery_discharge_limit_kWh / total_peak_periods if total_peak_periods > 0 else 0

# Calculate extra revenue from using the battery during peak times
battery_revenue = battery_discharge_per_period_kWh * building_price_cents / 100 * total_peak_periods

# Adjusted total revenue for Scenario 1
adjusted_revenue_scenario_1 = total_revenue_scenario_1 + battery_revenue

print(f"Scenario 1 Meter 216298730 Results:")
print(f"  - Total Revenue (no battery adjustment): ${total_revenue_scenario_1:.2f}")
print(f"  - Total Revenue (with battery adjustment): ${adjusted_revenue_scenario_1:.2f}")
print(f"  - Additional Revenue from Battery Usage: ${battery_revenue:.2f}")

# ------------------------------
# SCENARIO 2: Optimal Revenue Calculation
# ------------------------------

# Step 1: Determine the optimal price (higher of spot price or building price)
data['Optimal Price (cents/kWh)'] = data['Spot Price (cents/kWh)'].apply(
    lambda spot_price: max(spot_price, building_price_cents)
)

# Step 2: Calculate revenue from optimal grid sales (using chosen price)
data['Revenue from Grid (Optimal)'] = data['Excess Energy Exported to Grid'] * data['Optimal Price (cents/kWh)'] / 100

# Step 3: Total revenue for Scenario 2 includes:
# - Revenue from solar energy used by the building
# - Revenue from optimal grid sales
total_revenue_scenario_2 = data['Revenue from Grid (Optimal)'].sum() + total_solar_used_revenue

print(f"\nScenario 2 Results:")
print(f"  - Total Revenue (Optimal pricing): ${total_revenue_scenario_2:.2f}")


Scenario 1 Meter 216298730 Results:
  - Total Revenue (no battery adjustment): $10236.34
  - Total Revenue (with battery adjustment): $10246.95
  - Additional Revenue from Battery Usage: $10.61

Scenario 2 Results:
  - Total Revenue (Optimal pricing): $16981.35


In [16]:
data.to_csv("meter 1.csv",index=False)

# Meter 2

In [17]:
data = pd.read_csv('NewMeterWorkbook - 218691361.csv')
data.head()

,CUST,Start time for each 30min measurement period,Consumption in kWh,Trading Period,Price $/MWh for each trading period,Solar Energy Used,Excess Energy Exported to Grid,Revenue from Energy Used by Building,Revenue from Energy Exported to Grid,Total Revenue per Period
0,218691361,2023-01-01 00:00:00,0.10,1,3.24,0.10,3.23,0.0290,0.3876,0.4166
1,218691361,2023-01-01 00:30:00,1.01,2,4.36,1.01,2.32,0.2929,0.2784,0.5713
2,218691361,2023-01-01 01:00:00,0.67,3,3.61,0.67,2.66,0.1943,0.3192,0.5135
3,218691361,2023-01-01 01:30:00,0.11,4,6.13,0.11,3.22,0.0319,0.3864,0.4183
4,218691361,2023-01-01 02:00:00,0.09,5,4.74,0.09,3.24,0.0261,0.3888,0.4149


In [18]:

# Step 2: Convert time column to datetime format for better handling
data['Start time for each 30min measurement period'] = pd.to_datetime(
    data['Start time for each 30min measurement period']
)

# Constants for calculations
battery_capacity_kWh = 61  # Total battery capacity in kWh
battery_discharge_limit_kWh = battery_capacity_kWh * 0.6  # 60% discharge capacity
grid_price_cents = 12  # Price for selling energy to the grid (12 cents/kWh)
building_price_cents = 29  # Price for selling energy to the building (29 cents/kWh)

# Convert spot price from $/MWh to cents/kWh for easier comparison
data['Spot Price (cents/kWh)'] = data['Price $/MWh for each trading period'] / 10

# ------------------------------
# SCENARIO 1: Revenue Calculation
# ------------------------------

# Step 1: Calculate total revenue from solar energy used by the building
total_solar_used_revenue = data['Revenue from Energy Used by Building'].sum()

# Step 2: Calculate total revenue from excess solar energy exported to the grid
total_grid_export_revenue = data['Revenue from Energy Exported to Grid'].sum()

# Step 3: Combine these to get total revenue for Scenario 1
total_revenue_scenario_1 = total_solar_used_revenue + total_grid_export_revenue

# Step 4: Adjust for the impact of the 61kWh battery
# Peak times are defined as 6-9 AM and 6-9 PM (morning and evening periods)
peak_hours_morning = data[(data['Start time for each 30min measurement period'].dt.hour >= 6) &
                          (data['Start time for each 30min measurement period'].dt.hour < 9)]
peak_hours_evening = data[(data['Start time for each 30min measurement period'].dt.hour >= 18) &
                          (data['Start time for each 30min measurement period'].dt.hour < 21)]

# Calculate how much battery energy is available per peak period
total_peak_periods = len(peak_hours_morning) + len(peak_hours_evening)
battery_discharge_per_period_kWh = battery_discharge_limit_kWh / total_peak_periods if total_peak_periods > 0 else 0

# Calculate extra revenue from using the battery during peak times
battery_revenue = battery_discharge_per_period_kWh * building_price_cents / 100 * total_peak_periods

# Adjusted total revenue for Scenario 1
adjusted_revenue_scenario_1 = total_revenue_scenario_1 + battery_revenue

print(f"Scenario 1 Meter 218691361 Results:")
print(f"  - Total Revenue (no battery adjustment): ${total_revenue_scenario_1:.2f}")
print(f"  - Total Revenue (with battery adjustment): ${adjusted_revenue_scenario_1:.2f}")
print(f"  - Additional Revenue from Battery Usage: ${battery_revenue:.2f}")

# ------------------------------
# SCENARIO 2: Optimal Revenue Calculation
# ------------------------------

# Step 1: Determine the optimal price (higher of spot price or building price)
data['Optimal Price (cents/kWh)'] = data['Spot Price (cents/kWh)'].apply(
    lambda spot_price: max(spot_price, building_price_cents)
)

# Step 2: Calculate revenue from optimal grid sales (using chosen price)
data['Revenue from Grid (Optimal)'] = data['Excess Energy Exported to Grid'] * data['Optimal Price (cents/kWh)'] / 100

# Step 3: Total revenue for Scenario 2 includes:
# - Revenue from solar energy used by the building
# - Revenue from optimal grid sales
total_revenue_scenario_2 = data['Revenue from Grid (Optimal)'].sum() + total_solar_used_revenue

print(f"\nScenario 2 Results:")
print(f"  - Total Revenue (Optimal pricing): ${total_revenue_scenario_2:.2f}")


Scenario 1 Meter 218691361 Results:
  - Total Revenue (no battery adjustment): $8077.79
  - Total Revenue (with battery adjustment): $8088.41
  - Additional Revenue from Battery Usage: $10.61

Scenario 2 Results:
  - Total Revenue (Optimal pricing): $16549.02


In [19]:
data.to_csv("meter 2.csv",index=False)